###  Написать программу, которая собирает товары «В тренде» с сайта техники mvideo и складывает данные в БД. Сайт можно выбрать и свой. Главный критерий выбора: динамически загружаемые товары

In [46]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient
import pandas as pd

In [47]:
client = MongoClient('localhost', 27017)
db = client['selenium']
collection = db['mvidios']

In [48]:
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome(executable_path='../chromedriver.exe', options=chrome_options)
driver.implicitly_wait(10)
driver.get('https://www.mvideo.ru/')
wait = WebDriverWait(driver, 2)

<ipython-input-48-86f4f63c9d2c>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../chromedriver.exe', options=chrome_options)


In [49]:
while True:
    try:
        element = wait.until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(text(),'В тренде')]"))
        )
        element.click()
        break
    except:
        actions = ActionChains(driver)
        actions.key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN)
        actions.perform()

In [50]:
while True:
    try:

        next = wait.until(EC.element_to_be_clickable((By.XPATH, "//mvid-shelf-group/*/div[2]/button[2]")))
        next.click()
    except:
        break

In [51]:
elements = driver.find_elements(By.XPATH, '//mvid-shelf-group//a[@class="img-with-badge"]')
links = [i.get_attribute('href') for i in elements]
for i in links:
    position = {}
    driver.get(i)
    code = driver.find_element(By.CLASS_NAME, 'product-code-container').text
    name = driver.find_element(By.XPATH, '//h1[@class="title"]').text
    cost = driver.find_element(By.CLASS_NAME, 'price__main-value').text
    
    position["Код Товара"] = code
    position["Ссылка"] = i
    position["Цена"] = cost
    position["Название"] = name
    
    collection.update_one({'Ссылка': position['Ссылка']}, {'$set' : position}, upsert=True)

In [52]:
df = pd.DataFrame([i for i in collection.find({})])

In [53]:
df

,_id,Ссылка,Код Товара,Название,Цена
0,618af2d2205966064ab263be,https://www.mvideo.ru/products/noutbuk-acer-sw...,Код товара: 3005 7260,Ноутбук Acer Swift 1 SF114-34-C564 NX.A74ER.002,30 999 ₽
1,618af2d5205966064ab263c0,https://www.mvideo.ru/products/smartfon-huawei...,Код товара: 3005 3815,Смартфон Huawei P Smart 2021 4+128GB Crush Gre...,30 999 ₽
2,618af2d7205966064ab263c2,https://www.mvideo.ru/products/naushniki-nakla...,Код товара: 5015 0882,Наушники накладные Philips TAH2005BK/00,15 999 ₽
3,618af2d8205966064ab263c4,https://www.mvideo.ru/products/podpiska-mprime...,Код товара: 6016151,Подписка M.Prime на 1 месяц + Яндекс.Плюс,499 ₽
4,618af2d9205966064ab263c7,https://www.mvideo.ru/products/konvektor-elect...,Код товара: 2007 4933,Конвектор Electrolux ECH/U-2000 ER,6 999 ₽
5,618af2db205966064ab263cb,https://www.mvideo.ru/products/smart-chasy-sam...,Код товара: 3005 1392,Смарт-часы Samsung Galaxy Watch3 45mm Черные (...,6 999 ₽
6,618af2dc205966064ab263cd,https://www.mvideo.ru/products/elektrogril-del...,Код товара: 2007 5969,Электрогриль DeLonghi CGH920D,25 499 ₽
7,618af2dd205966064ab263d0,https://www.mvideo.ru/products/elektrochainik-...,Код товара: 2007 4031,Электрочайник Tefal Sense KO693110,9 999 ₽
8,618af2df205966064ab263d2,https://www.mvideo.ru/products/kofemashina-kap...,Код товара: 2002 9115,Кофемашина капсульного типа DeLonghi INISSIA E...,4 499 ₽
9,618af2e0205966064ab263d4,https://www.mvideo.ru/products/ruchnoi-otpariv...,Код товара: 2007 2378,Ручной отпариватель Philips STH3020/10,5 999 ₽
